# Wind Machine Learning Model
This notebook creates a machine learning model 

In [2]:
# Initial Imports
import requests
import pandas as pd
import json
import numpy as np
from path import Path
from datetime import datetime
from datetime import timedelta
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import keras

ModuleNotFoundError: No module named 'keras'

## Linear Regression

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [ ]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [ ]:
# Visually inspect the data
plt.scatter(hackberry.WindSpeed_mph, hackberry.MWH)
plt.xlabel('Wind Speed')
plt.ylabel('MHW')
plt.show()

In [ ]:
# Visually inspect the data
plt.scatter(hackberry.WindDirection_degrees, hackberry.MWH)
plt.xlabel('Wind Direction')
plt.ylabel('MHW')
plt.show()

In [ ]:
# Visually inspect the data
plt.scatter(hackberry.WindGust_mph, hackberry.MWH)
plt.xlabel('Wind Gust')
plt.ylabel('MHW')
plt.show()

##  It can be concluded that linear regression on wind speed alone does not track the power generated.  Direction of wind needs to be included.

In [ ]:
# Creating dataframe with wind direction blowing east
windEast = hackberry[hackberry.WindDirection_compass == 'E']

In [ ]:
# Visually inspect the data of wind speed to MWH for winds blowing east.
plt.scatter(windEast.WindSpeed_mph, windEast.MWH)
plt.xlabel('Wind Speed')
plt.ylabel('MWH')
plt.show()

In [ ]:
# Creating dataframe with wind direction blowing West
windWest = hackberry[hackberry.WindDirection_compass == 'W']

In [ ]:
# Visually inspect the data of wind speed to MWH for winds blowing West.
plt.scatter(windWest.WindSpeed_mph, windWest.MWH)
plt.xlabel('Wind Speed')
plt.ylabel('MWH')
plt.show()

In [ ]:
# Creating dataframe with wind direction blowing south
windSouth = hackberry[hackberry.WindDirection_compass == 'S']

In [ ]:
# Visually inspect the data of wind speed to MWH for winds blowing South.
plt.scatter(windSouth.WindSpeed_mph, windSouth.MWH)
plt.xlabel('Wind Speed')
plt.ylabel('MWH')
plt.show()

In [ ]:
# Creating dataframe with wind direction blowing North
windNorth = hackberry[hackberry.WindDirection_compass == 'N']

In [ ]:
# Visually inspect the data of wind speed to MWH for winds blowing North.
plt.scatter(windNorth.WindSpeed_mph, windNorth.MWH)
plt.xlabel('Wind Speed')
plt.ylabel('MWH')
plt.show()

In [ ]:
# Creating dataframe with wind direction blowing NE
windNE = hackberry[hackberry.WindDirection_compass == 'NE']

In [ ]:
# Visually inspect the data of wind speed to MWH for winds blowing NE.
plt.scatter(windNE.WindSpeed_mph, windNE.MWH)
plt.xlabel('Wind Speed')
plt.ylabel('MWH')
plt.show()

In [ ]:
# Creating dataframe with wind direction blowing NW
windNW = hackberry[hackberry.WindDirection_compass == 'NW']

In [ ]:
# Visually inspect the data of wind speed to MWH for winds blowing NW.
plt.scatter(windNW.WindSpeed_mph, windNW.MWH)
plt.xlabel('Wind Speed')
plt.ylabel('MWH')
plt.show()

In [ ]:
# Creating dataframe with wind direction blowing NNE
windNNE = hackberry[hackberry.WindDirection_compass == 'NNE']

In [ ]:
# Visually inspect the data of wind speed to MWH for winds blowing NNE.
plt.scatter(windNNE.WindSpeed_mph, windNNE.MWH)
plt.xlabel('Wind Speed')
plt.ylabel('MWH')
plt.show()

In [ ]:
# Creating dataframe with wind direction blowing NNW
windNNW = hackberry[hackberry.WindDirection_compass == 'NNW']

In [ ]:
# Visually inspect the data of wind speed to MWH for winds blowing NNW.
plt.scatter(windNNW.WindSpeed_mph, windNNW.MWH)
plt.xlabel('Wind Speed')
plt.ylabel('MWH')
plt.show()

In [ ]:
# Creating a dictionary with wind direction
#directions = {
#    "N": 1,
#   "S": 2,
#   "E": 3,
#   "W": 4,
#   "NE": 5,
#  "NW": 6,
#   "NNE": 7,
#   "NNW": 8,
#   "SE": 11,
#   "SW": 12,
#   "SSE": 13,
#    "SSW":14,
#    "WNW":15,
#    "WSW":16,
#    "ENE":17,
#    "ESE":18,
    
#}

In [ ]:
#hackberry["direction"] = hackberry["WindDirection_compass"].apply(lambda x: directions[x])

In [ ]:
hackberry.to_csv(r'Output/hackberry_encoded.csv', index = False)

In [66]:
data = "Output/hackberry_encoded.csv"
hackberry_encoded = pd.read_csv(data)
hackberry_encoded

,time,WindSpeed_mph,WindDirection_degrees,WindDirection_compass,WindGust_mph,MWH
0,2019-01-01 01:00:00,13.0,89.0,E,23.0,110.487950
1,2019-01-01 02:00:00,14.0,53.0,NE,23.0,72.020225
2,2019-01-01 03:00:00,15.0,17.0,NNE,22.0,67.639475
3,2019-01-01 04:00:00,14.0,18.0,NNE,21.0,63.718900
4,2019-01-01 05:00:00,14.0,19.0,NNE,20.0,61.264250
...,...,...,...,...,...,...
8753,2019-12-31 19:00:00,6.0,175.0,S,12.0,15.506725
8754,2019-12-31 20:00:00,6.0,176.0,S,13.0,32.191125
8755,2019-12-31 21:00:00,7.0,176.0,S,14.0,40.677250
8756,2019-12-31 22:00:00,7.0,176.0,S,15.0,45.826475


In [ ]:
#hackberryEncoded = hackberry_encoded.drop(["time", "WindDirection_compass", "direction"], axis=1)

In [67]:
hackberryEncoded = hackberry_encoded.drop([ "WindDirection_compass", "WindDirection_compass"], axis=1)

In [68]:
hackberryEncoded["WindSpeed_mph"] = hackberryEncoded["WindSpeed_mph"].round(0).astype(int)

In [69]:
hackberryEncoded["WindDirection_degrees"] = hackberryEncoded["WindDirection_degrees"].round(0).astype(int)

In [70]:
hackberryEncoded["WindGust_mph"] = hackberryEncoded["WindGust_mph"].round(0).astype(int)

In [71]:
hackberryEncoded["MWH"] = hackberryEncoded["MWH"].round(0).astype(int)

In [77]:
import datetime as dt
hackberryEncoded['time'] = pd.to_datetime(hackberryEncoded['time'])
hackberryEncoded['time'] = hackberryEncoded['time'].dt.hour


In [94]:
hackberryEncoded.dtypes

time                     int64
WindSpeed_mph            int32
WindDirection_degrees    int32
WindGust_mph             int32
MWH                      int32
dtype: object

In [55]:
# Creating the scaler instance
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()

In [34]:
hackberry_scaled = data_scaler.fit_transform(hackberryEncoded)

TypeError: invalid type promotion

In [13]:
hackberry_scaled[:5]

array([[ 0.3608004 , -1.05022902,  0.74188853,  1.22371389],
       [ 0.56803347, -1.53116107,  0.74188853,  0.40867688],
       [ 0.77526654, -2.01209313,  0.60454765,  0.32288351],
       [ 0.56803347, -1.99873391,  0.46720677,  0.23709014],
       [ 0.56803347, -1.98537468,  0.32986589,  0.17274512]])

In [ ]:
print(np.mean(hackberry_scaled[:,0]))
print(np.std(hackberry_scaled[:,0]))

In [82]:
# Define the features set.
X = hackberryEncoded.copy()
X = X.drop("MWH", axis=1)
X.head()

,time,WindSpeed_mph,WindDirection_degrees,WindGust_mph
0,1,13,89,23
1,2,14,53,23
2,3,15,17,22
3,4,14,18,21
4,5,14,19,20


In [83]:
# Define the target set.
y = hackberryEncoded["MWH"].ravel()
y[:5]

array([110,  72,  68,  64,  61])

In [84]:
#split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [85]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [60]:
# Determine the shape of training and testing sets.
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(6568, 2)
(2190, 2)
(6568,)
(2190,)


# ML Models

## Multiple Linear Regression

Date time not supported in linear Regression.

In [86]:
from sklearn.linear_model import LinearRegression
# Train

regr = LinearRegression()
regr.fit(X_train,y_train)


LinearRegression()

In [87]:
# test
y_pred = regr.predict(X_test)
y_pred

array([ -1.94027559,  35.48290663,  46.02611518, ...,  37.05014937,
        91.5921314 , 117.81088724])

In [88]:
accuracy = regr.score(X_test,y_test)
print(accuracy*100,'%')
print(f"R^2 Value:{regr.score(X_test,y_test)}")


38.46475037237571 %
R^2 Value:0.3846475037237571


## Neural Network

In [102]:
import tensorflow as tf
# Define the model - deep neural net
number_input_features = 4
hidden_nodes_layer1 =  number_input_features*3
hidden_nodes_layer2 =  number_input_features*3

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="linear")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="linear"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="linear"))

# Compile the Sequential model together and customize metrics
nn.compile(optimizer="sgd", loss=tf.keras.losses.MeanSquaredError(), metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
206/206 [==============================] - 0s 573us/step - loss: 3781.8059 - accuracy: 0.0467
Epoch 2/50
206/206 [==============================] - 0s 573us/step - loss: 1960.5571 - accuracy: 0.0437
Epoch 3/50
206/206 [==============================] - 0s 670us/step - loss: 1671.8752 - accuracy: 0.0437
Epoch 4/50
206/206 [==============================] - 0s 961us/step - loss: 1479.2192 - accuracy: 0.0437
Epoch 5/50
206/206 [==============================] - 0s 859us/step - loss: 1401.5864 - accuracy: 0.0437
Epoch 6/50
206/206 [==============================] - 0s 607us/step - loss: 1389.9894 - accuracy: 0.0438
Epoch 7/50
206/206 [==============================] - 0s 573us/step - loss: 1382.8456 - accuracy: 0.0443
Epoch 8/50
206/206 [==============================] - 0s 592us/step - loss: 1380.9604 - accuracy: 0.0443
Epoch 9/50
206/206 [==============================] - 0s 563us/step - loss: 1381.0540 - accuracy: 0.0446
Epoch 10/50
206/206 [==============================] - 

In [107]:
# Example from https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/
import keras
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score, KFold
from keras.models import Sequential
from sklearn.metrics import accuracy_score
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
seed = 1

# diabetes = datasets.load_diabetes()
# X = diabetes.data[:150]
# y = diabetes.target[:150]

def baseline_model():
    model = Sequential()
    model.add(Dense(10, input_dim=4, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model


# estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=100, batch_size=100, verbose=False)
# kfold = KFold(n_splits=10, random_state=seed)
# results = cross_val_score(estimator, X, y, cv=kfold)
# print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

# estimator.fit(X, y)
# prediction = estimator.predict(X)
# accuracy_score(y, prediction)

estimator = KerasRegressor(build_fn=baseline_model)
estimator.fit(X_train, y_train, nb_epoch=100, batch_size=100, verbose=False, shuffle=False)
prediction = estimator.predict(X_test)


# model = baseline_model()
# model.fit(X, y, nb_epoch=100, batch_size=100, verbose=False, shuffle=False)
# prediction = model.predict(X)

ModuleNotFoundError: No module named 'keras'

## Random Forrest

In [ ]:
# Resample the training data with the RandomOversampler
from imblearn.ensemble import BalancedRandomForestClassifier
brfc = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brfc.fit(X_train, y_train)
Counter(y_train)

In [ ]:
# Calculated the balanced accuracy score
y_pred = brfc.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
# List the features sorted in descending order by feature importance
importances = brfc.feature_importances_
sorted(zip(brfc.feature_importances_, X.columns), reverse=True)